ran on kaggle

# Baseline TFT for Ontario Demand using engineered parquet
# - Target: "Ontario Demand"
# - Uses precomputed features from /ontario_demand_basic_features.parquet
# - Encoder: 168h (7 days), Prediction length: 24h

# 0) Installs (safe on Kaggle)
!pip -q install "pytorch-forecasting==1.4.0" "lightning>=2.2,<2.5" "torchmetrics>=1.3,<1.5" --no-cache-dir
 
# ==============================
# 1) Imports and seed
# ==============================
import os, math
import numpy as np
import pandas as pd
import torch

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping

from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss

pl.seed_everything(42, workers=True)

# ==============================
# 2) Load engineered parquet
# ==============================
# If your Kaggle dataset mounts elsewhere, adjust the path accordingly.
parquet_path = "/kaggle/input/ontario-demand-basic-features"
df = pd.read_parquet(parquet_path)

# Clean column names lightly (keep spaces for target)
df.columns = [c.strip() for c in df.columns]

# Sanity checks
assert "Ontario Demand" in df.columns, "Expected 'Ontario Demand' column"
assert "time_idx" in df.columns, "Expected 'time_idx' column"
assert "series_id" in df.columns, "Expected 'series_id' column"

# Ensure dtypes/order
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

df = df.sort_values("time_idx").reset_index(drop=True)
df["Ontario Demand"] = pd.to_numeric(df["Ontario Demand"], errors="coerce")

# ==============================
# 3) Feature sets
# ==============================
# Known in future (calendar + harmonics + flags; adjust as needed)
known_reals = [
    "time_idx",
    "hour", "dow", "day", "week", "month", "quarter", "dayofyear",
    "is_weekend", "is_business_day", "is_holiday", "is_holiday_eve", "is_holiday_morrow",
    "season_code", "is_month_start", "is_month_end", "is_qtr_end", "is_year_end",
    "is_dst", "is_peak_hour",
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "doy_sin", "doy_cos",
]

# Unknown in future (encoder-only observed inputs: target + lags/rolls/ewm)
unknown_reals = [
    "Ontario Demand",
    "Ontario Demand_lag_1h", "Ontario Demand_lag_24h", "Ontario Demand_lag_168h",
    "Ontario Demand_rollmean_24h", "Ontario Demand_rollstd_24h",
    "Ontario Demand_rollmean_168h", "Ontario Demand_rollstd_168h",
    "Ontario Demand_ewm_24h", "Ontario Demand_ewm_168h",
]

# Keep only existing columns (handles cases where some features are missing)
known_reals = [c for c in known_reals if c in df.columns]
unknown_reals = [c for c in unknown_reals if c in df.columns]

# Optional: drop rows with missing target
df = df.dropna(subset=["Ontario Demand"]).reset_index(drop=True)

# ==============================
# 4) Split and dataset
# ==============================
max_encoder_length = 168   # 7 days
max_prediction_length = 24 # next 24 hours

# Chronological split: last 7 days for validation
cutoff = df["time_idx"].max() - max_prediction_length * 7


max_history = 168  # largest lookback among *_lag_168h / *_roll*_168h / *_ewm_168h
min_time_per_series = df.groupby("series_id")["time_idx"].transform("min")
df = df[df["time_idx"] >= (min_time_per_series + max_history)].copy()
df = df.sort_values("time_idx").reset_index(drop=True)


training_df = df[df["time_idx"] <= cutoff].copy()
validation_df = df[df["time_idx"] > cutoff].copy()
assert len(training_df) > 0 and len(validation_df) > 0

training = TimeSeriesDataSet(
    training_df,
    time_idx="time_idx",
    target="Ontario Demand",
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,

    time_varying_known_reals=known_reals,
    time_varying_unknown_reals=unknown_reals,

    # Normalization and helpers
    target_normalizer=GroupNormalizer(groups=["series_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,

    # Tolerate any residual gaps
    allow_missing_timesteps=True,
)

# Build validation dataset aligned to the training definition
validation = TimeSeriesDataSet.from_dataset(
    training, df, predict=True, stop_randomization=True
)

# ==============================
# 5) Dataloaders
# ==============================
batch_size = 128
train_loader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0, persistent_workers=False
)
val_loader = validation.to_dataloader(
    train=False, batch_size=batch_size * 2, num_workers=0, persistent_workers=False
)

# ==============================
# 6) Model and trainer
# ==============================
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=1e-3,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    loss=QuantileLoss(),      # robust multi-horizon baseline
    optimizer="adam",
    reduce_on_plateau_patience=3,
)

early_stop = EarlyStopping(monitor="val_loss", patience=3, mode="min")

trainer = pl.Trainer(
    max_epochs=5,             # quick baseline
    accelerator="auto",
    devices="auto",
    gradient_clip_val=0.1,
    callbacks=[early_stop],
    log_every_n_steps=50,
)

# ==============================
# 7) Train
# ==============================
trainer.fit(tft, train_loader, val_loader)

# ==============================
# 8) Validation predictions & metrics (unchanged)
# ==============================
preds = tft.predict(val_loader)  # [N_windows, 24]

# Collect actual targets from val_loader by unpacking y=(target, weight)
actuals_list = []
for _, y in val_loader:
    target = y[0] if isinstance(y, (list, tuple)) else y
    if isinstance(target, list):
        target = target[0]
    actuals_list.append(target)

actuals = torch.cat([t.detach().cpu().float() for t in actuals_list], dim=0)
preds = preds.detach().cpu().float()

# Overall metrics
mae = torch.mean(torch.abs(actuals - preds)).item()
rmse = torch.sqrt(torch.mean((actuals - preds) ** 2)).item()
smape = torch.mean(200.0 * torch.abs(actuals - preds) / (torch.abs(actuals) + torch.abs(preds) + 1e-6)).item()
print(f"Validation MAE:  {mae:.2f}")
print(f"Validation RMSE: {rmse:.2f}")
print(f"Validation sMAPE: {smape:.2f}%")

# Per-horizon metrics (1..24)
per_h_mae = torch.mean(torch.abs(actuals - preds), dim=0).numpy()
per_h_rmse = torch.sqrt(torch.mean((actuals - preds) ** 2, dim=0)).numpy()
per_h_smape = torch.mean(200.0 * torch.abs(actuals - preds) / (torch.abs(actuals) + torch.abs(preds) + 1e-6), dim=0).numpy()

import pandas as pd
per_h_df = pd.DataFrame({
    "horizon_hour_ahead": np.arange(1, preds.shape[1] + 1),
    "MAE": per_h_mae,
    "RMSE": per_h_rmse,
    "sMAPE_%": per_h_smape,
})
print(per_h_df.head(10))
per_h_df.to_csv("/kaggle/working/validation_per_horizon_metrics.csv", index=False)

# Optional: inspect first validation window
sample_idx = 0
inspect_df = pd.DataFrame({
    "horizon_hour_ahead": np.arange(1, preds.shape[1] + 1),
    "pred_p50": preds[sample_idx].numpy(),
    "actual": actuals[sample_idx].numpy(),
})
print(inspect_df.head(12))
inspect_df.to_csv("/kaggle/working/sample_validation_window.csv", index=False)

print("Artifacts saved to /kaggle/working/: validation_per_horizon_metrics.csv, sample_validation_window.csv")


Validation MAE:  558.37
Validation RMSE: 636.15
Validation sMAPE: 3.82%